In [ ]:
#r "nuget: Plotly.NET, *-*"
#r "nuget: Plotly.NET.Interactive, *-*"
#r "nuget: FSharp.Stats"
#r "nuget: Deedle"


Installed Packages Deedle, 2.4.2 FSharp.Stats, 0.4.2 Plotly.NET, 2.0.0-preview.7 Plotly.NET.Interactive, 2.0.0-preview.7

Loading extensions from `Plotly.NET.Interactive.dll`

Added Kernel Extension including formatters for Plotly.NET charts.

# Basic linear regression plots

In [ ]:
open Deedle
open FSharp.Data
open FSharp.Stats
open FSharp.Stats.Fitting.LinearRegression
open Plotly.NET

let data=
    Http.RequestString "https://raw.githubusercontent.com/plotly/datasets/master/tips.csv"
    |> fun csv -> Frame.ReadCsvString(csv,true,separators=",")
    

let getColumnData column=
        data
        |> Frame.getCol column
        |> Series.values
        |> Array.ofSeq

let y = vector <| getColumnData "tip"
let x = vector <| getColumnData "total_bill"

let coefs = OrdinaryLeastSquares.Linear.Univariable.coefficient x y
let fittinFunc x= OrdinaryLeastSquares.Linear.Univariable.fit coefs x

let xRange = [for i in Seq.min(x)..Seq.max(x) -> i]
let yPredicted = [for x in xRange -> fittinFunc x]

let xy = Seq.zip xRange yPredicted
[
    Chart.Point(x,y,Showlegend=true,Name="Tips")
    |> Chart.withXAxisStyle(title="total_bill")
    |> Chart.withYAxisStyle(title="tip");

    Chart.Line(xy,Showlegend=true,Name="Regression Fit")
]
|> Chart.combine



# Model generalization on unseen data

In [ ]:
open Deedle
open FSharp.Data
open FSharp.Stats
open FSharp.Stats.Fitting
open FSharp.Stats.Fitting.LinearRegression
open Plotly.NET

let data=
    Http.RequestString "https://raw.githubusercontent.com/plotly/datasets/master/tips.csv"
    |> fun csv -> Frame.ReadCsvString(csv,true,separators=",")
    

let getColumnData column=
        data
        |> Frame.getCol column
        |> Series.values
        |> Array.ofSeq

let y = vector <| getColumnData "tip"
let x = vector <| getColumnData "total_bill"

let splitPercetage = 0.2
let n = x.NumRows
let m = float n * splitPercetage |> ceil |> int

let chunkIndices =
                [|0 .. n-1|]
                |> FSharp.Stats.Array.shuffleFisherYates
                |> Array.take m

let xTest,xTrain = x |> Vector.splitVector chunkIndices
let yTest,yTrain = y |> Vector.splitVector chunkIndices

let coefs = OrdinaryLeastSquares.Linear.Univariable.coefficient xTrain yTrain
let fittinFunc x= OrdinaryLeastSquares.Linear.Univariable.fit coefs x

let xRange = [for i in Seq.min(x)..((Seq.max(x)-Seq.min(x))/100.)..Seq.max(x) -> i]
let yPredicted = [for x in xRange -> fittinFunc x]

let xy = Seq.zip xRange yPredicted

[
    Chart.Point(xTrain,yTrain,Showlegend=true,Name="train")
    |> Chart.withXAxisStyle(title="total_bill")
    |> Chart.withYAxisStyle(title="tip");

    Chart.Point(xTest,yTest,Showlegend=true,Name="test")

    Chart.Line(xy,Showlegend=true,Name="Regression Fit")
]
|> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->

# Comparing different models parameters

In [ ]:
open Deedle
open FSharp.Data
open FSharp.Stats
open FSharp.Stats.Fitting.LinearRegression
open Plotly.NET

let data=
    Http.RequestString "https://raw.githubusercontent.com/plotly/datasets/master/tips.csv"
    |> fun csv -> Frame.ReadCsvString(csv,true,separators=",")
    

let getColumnData column=
        data
        |> Frame.getCol column
        |> Series.values
        |> Array.ofSeq

let y = vector <| getColumnData "tip"
let x = vector <| getColumnData "total_bill"

let order = 5

let coefs = OrdinaryLeastSquares.Polynomial.coefficient order x y
let fittinFunc x= OrdinaryLeastSquares.Polynomial.fit order coefs x

let xRange = [for i in Seq.min(x)..((Seq.max(x)-Seq.min(x))/100.)..Seq.max(x) -> i]
let yPredicted = [for x in xRange -> fittinFunc x]

let weights = y |> Vector.map (fun y -> 1. / y)
let coefsWeighted = OrdinaryLeastSquares.Polynomial.coefficientsWithWeighting order weights x y
let fittingWeightedFunc x= OrdinaryLeastSquares.Polynomial.fit order coefsWeighted x

let yPredictedWeighted = [for x in xRange -> fittingWeightedFunc x]

let xy = Seq.zip xRange yPredicted
let xyWeighted = Seq.zip xRange yPredictedWeighted
[
    Chart.Point(x,y,Showlegend=true,Name="Tips")
    |> Chart.withXAxisStyle(title="total_bill")
    |> Chart.withYAxisStyle(title="tip");

    Chart.Line(xy,Showlegend=true,Name="Polynomial Fit");
    Chart.Line(xyWeighted,Showlegend=true,Name="Weighted Polynomial Fit")
]
|> Chart.combine


<!-- Plotly chart will be drawn inside this DIV -->

# 3D regression surface